# Some utility functions

In [67]:
def create_directed_adjacency_list(edges):
  vertices = set()
  for u, v, w in edges:
    vertices.add(u)
    vertices.add(v)

  adjacency_list = {}
  for v in vertices:
    adjacency_list[v] = []
  for u, v, w in edges:
    adjacency_list[u].append((v, w))  # u -> v
  return adjacency_list


def create_undirected_adjacency_list(edges):
  vertices = set()
  for u, v, w in edges:
    vertices.add(u)
    vertices.add(v)

  adjacency_list = {}
  for v in vertices:
    adjacency_list[v] = []
  for u, v, w in edges:
    adjacency_list[u].append((v, w))  # u -> v (undirected graph)
    adjacency_list[v].append((u, w))  # v -> u (undirected graph)
  return adjacency_list


create_directed_adjacency_list([('A', 'B', 10), ('B', 'H', 20)])
create_undirected_adjacency_list([('A', 'B', 10), ('B', 'H', 20)])

{'A': [('B', 10)], 'B': [('H', 20)], 'H': []}

{'A': [('B', 10)], 'B': [('A', 10), ('H', 20)], 'H': [('B', 20)]}

# GrPA 1

## Solution Approach: FiberLink

The goal is to connect `n` cities with minimum fiber length using Kruskal's Algorithm.

### Overview

To solve this problem, we'll:
1. Use Kruskal's Algorithm to build the Minimum Spanning Tree (MST).
2. Sum the weights of the MST edges to get the total fiber length required.

### Steps

1. **Kruskal's Algorithm**:
   - **Initialize**: 
     - `mst`: List to store the Minimum Spanning Tree (MST) edges.
     - `edges`: List to store all edges in the graph.
     - `parent`: Dictionary for the Union-Find structure, each vertex is its own parent.
   - **Populate Edges**:
     - Iterate through `adjacency_list`.
     - For each vertex and its neighbors, add `(weight, vertex, neighbor)` to `edges`.
   - **Sort Edges**:
     - Sort `edges` based on weights.
   - **Union-Find**:
     - Define `find(node)` to get the root parent.
     - For each edge `(weight, u, v)` in sorted `edges`:
       - Find root parents `root1` and `root2` of `u` and `v`.
       - If roots are different, merge them (`parent[root2] = root1`) and add edge to `mst`.
   - **Return**:
     - The MST edges.

2. **FiberLink Function**:
   - **Compute MST**:
     - Call `kruskal(adjacency_list)` to get `mst`.
   - **Calculate Total Fiber Length**:
     - Initialize `fiber_length` to 0.
     - Sum the weights of all edges in `mst`.
   - **Return**:
     - The total fiber length.

In [68]:
# Kruskal's Algorithm (standard implementation)
def kruskal(adjacency_list):
  mst = []

  edges = []
  for vertex in adjacency_list:
    for neighbor, weight in adjacency_list[vertex]:
      edges.append((weight, vertex, neighbor))
  edges.sort()

  parent = {vertex: vertex for vertex in adjacency_list}

  def find(node):
    while parent[node] != node:
      node = parent[node]
    return node

  for weight, u, v in edges:
    root1 = find(u)
    root2 = find(v)
    if root1 != root2:
      parent[root2] = root1
      mst.append((u, v, weight))

  return mst

In [69]:
def fiber_link(adjacency_list):
  mst = kruskal(adjacency_list)  # Kruskal's Algorithm (standard implementation)
  fiber_length = 0
  for u, v, w in mst:
    fiber_length += w
  return fiber_length

In [70]:
# Test cases

edges = [(0, 1, 10), (0, 2, 50), (0, 3, 300), (5, 6, 45), (2, 1, 30),
         (6, 4, 37), (1, 6, 65), (2, 5, 76), (1, 3, 40), (3, 4, 60), (2, 4, 20)]
fiber_link(create_undirected_adjacency_list(edges)) == 182

edges = [(0, 1, 1), (0, 2, 6), (1, 2, 3), (1, 3, 4), (2, 4, 4), (2, 3, 2),
         (3, 4, 3), (1, 5, 2), (2, 5, 7), (3, 5, 1), (4, 5, 5)]
fiber_link(create_undirected_adjacency_list(edges)) == 9

edges = [(0, 1, 16), (0, 3, 2), (1, 2, 4), (3, 4, 10), (0, 4, 9), (3, 5, 15), (1, 5, 7), (2, 5, 6)]
fiber_link(create_undirected_adjacency_list(edges)) == 36

edges = [(0, 1, 10), (0, 2, 4), (1, 2, 7), (0, 5, 4), (0, 4, 2), (2, 5, 7), (3, 5, 2), (3, 4, 6)]
fiber_link(create_undirected_adjacency_list(edges)) == 19

edges = [(0, 1, 1), (0, 2, 4), (1, 2, 7), (0, 5, 14), (0, 4, 12), (2, 5, 17), (3, 5, 22), (3, 4, 26)]
fiber_link(create_undirected_adjacency_list(edges)) == 53

True

True

True

True

True

# GrPA 2

## Solution Approach: MinCostWalk

The goal is to find the minimum cost walk from node `S` to node `D` via node `V` using Dijkstra's Algorithm.

### Overview

To solve this problem, we'll:
1. Use Dijkstra's Algorithm to find the shortest path from `S` to `V` and from `V` to `D`.
2. Combine these paths to get the total minimum cost and the walk route.

### Steps

1. **Priority Queue Class**:
   - Implement a `PriorityQueue` using `heapq` to manage vertices based on distance.

2. **Dijkstra's Algorithm**:
   - **Initialize**:
     - `visited`: Dictionary to track visited vertices.
     - `predecessor`: Dictionary to store the path.
     - `distance`: Dictionary to store the shortest distance from the start vertex.
   - **Processing**:
     - Start from `start_vertex`, set its distance to 0, and enqueue it.
     - While the queue is not empty:
       - Dequeue the vertex with the smallest distance.
       - For each neighbor, calculate the new distance.
       - If the new distance is shorter, update the distance and predecessor, and enqueue the neighbor.

3. **Construct Path**:
   - Use the `predecessor` dictionary to backtrack from the destination to the start to get the path.

4. **MinCostWalk Function**:
   - **Compute Paths**:
     - Use Dijkstra's Algorithm to find the shortest path and distance from `S` to `V` (`dist_S`, `predecessor_S`).
     - Use Dijkstra's Algorithm to find the shortest path and distance from `V` to `D` (`dist_V`, `predecessor_V`).
   - **Combine Paths**:
     - Construct the path from `S` to `V` (`path_S`).
     - Construct the path from `V` to `D` (`path_V`).
     - Merge these paths, ensuring not to repeat node `V`.
   - **Calculate Total Distance**:
     - Sum the distances from `S` to `V` and from `V` to `D`.
   - **Return**:
     - The total minimum cost and the combined walk route.

In [71]:
import heapq


class PriorityQueue:
  def __init__(self):
    self.heap = []

  def is_empty(self):
    return len(self.heap) == 0

  def enqueue(self, priority, item):
    heapq.heappush(self.heap, (priority, item))

  def dequeue(self):
    return heapq.heappop(self.heap)[1]


pq = PriorityQueue()
pq.enqueue(4, 'B')
pq.enqueue(2, 'C')
print(pq.dequeue())
print(pq.dequeue())

C
B


In [72]:
# Dijkstra's Algorithm (slightly modified for `predecessor`)
def dijkstra(adjacency_list, start_vertex):
  visited = {vertex: False for vertex in adjacency_list}
  predecessor = {vertex: None for vertex in adjacency_list}  # NEW 😱
  distance = {vertex: float('inf') for vertex in adjacency_list}
  distance[start_vertex] = 0

  pq = PriorityQueue()
  pq.enqueue(0, start_vertex)

  while not pq.is_empty():
    current_vertex = pq.dequeue()
    visited[current_vertex] = True

    # print(f'{current_vertex=}')

    for neighbor, weight in adjacency_list[current_vertex]:
      if not visited[neighbor]:
        new_distance = distance[current_vertex] + weight
        if new_distance < distance[neighbor]:
          distance[neighbor] = new_distance
          pq.enqueue(new_distance, neighbor)
          predecessor[neighbor] = current_vertex  # NEW 😱

  return distance, predecessor


adjacency_list = {
    'A': [('B', 4), ('C', 2)],
    'B': [('C', 3), ('D', 2), ('E', 3)],
    'C': [('B', 1), ('D', 4), ('E', 5)],
    'D': [],
    'E': [('D', 1)],
}

dijkstra(adjacency_list, 'A')  # {'A': 0, 'B': 3, 'C': 2, 'D': 5, 'E': 6}

({'A': 0, 'B': 3, 'C': 2, 'D': 5, 'E': 6},
 {'A': None, 'B': 'C', 'C': 'A', 'D': 'B', 'E': 'B'})

In [73]:
def construct_path(predecessor, destination):
  path = []
  while destination is not None:
    path.append(destination)
    destination = predecessor[destination]
  path.reverse()
  return path


def min_cost_walk(adjacency_list, S, D, V):  # go: S -> V -> D
  dist_S, predecessor_S = dijkstra(adjacency_list, S)
  dist_V, predecessor_V = dijkstra(adjacency_list, V)

  path_S = construct_path(predecessor_S, V)  # S -> V
  path_V = construct_path(predecessor_V, D)  # V -> D

  # print(f'{dist_S=}, {predecessor_S=}, {path_S=}')
  # print(f'{dist_V=}, {predecessor_V=}, {path_V=}')

  total_distance = dist_S[V] + dist_V[D]
  walk_route = path_S[:-1] + path_V
  return total_distance, walk_route


edges = [(0, 1, 10), (0, 2, 50), (0, 3, 300), (5, 6, 45), (2, 1, 30),
         (6, 4, 37), (1, 6, 65), (2, 5, 76), (1, 3, 40), (3, 4, 60), (2, 4, 20)]
min_cost_walk(create_undirected_adjacency_list(edges), 0, 4, 5)  # (198, [0, 1, 2, 5, 6, 4])

(198, [0, 1, 2, 5, 6, 4])

In [74]:
# Test cases

edges = [(0, 1, 10), (0, 2, 50), (0, 3, 300), (5, 6, 45), (2, 1, 30),
         (6, 4, 37), (1, 6, 65), (2, 5, 76), (1, 3, 40), (3, 4, 60), (2, 4, 20)]
min_cost_walk(create_undirected_adjacency_list(edges), 0, 5, 1) == (116, [0, 1, 2, 5])

edges = [(0, 1, 10), (0, 2, 20), (0, 3, 30), (5, 6, 120), (2, 1, 5), (6, 4, 20),
         (1, 6, 15), (2, 5, 70), (1, 3, 7), (3, 4, 100), (2, 4, 50)]
min_cost_walk(create_undirected_adjacency_list(edges), 2, 3, 4) == (82, [2, 1, 6, 4, 6, 1, 3])

edges = [(0, 1, 10), (0, 2, 20), (0, 3, 30), (5, 6, 120), (2, 1, 5), (6, 4, 20),
         (1, 6, 15), (2, 5, 70), (1, 3, 7), (3, 4, 100), (2, 4, 50)]
min_cost_walk(create_undirected_adjacency_list(edges), 0, 4, 1) == (45, [0, 1, 6, 4])

edges = [(0, 1, 10), (0, 2, 20), (0, 3, 30), (5, 6, 120), (2, 1, 5), (6, 4, 20),
         (1, 6, 15), (2, 5, 70), (1, 3, 7), (3, 4, 100), (2, 4, 50)]
min_cost_walk(create_undirected_adjacency_list(edges), 0, 4, 5) == (195, [0, 1, 2, 5, 2, 1, 6, 4])

True

True

True

True

# GrPA 3

## Solution Approach: IsNegativeWeightCyclePresent

The goal is to detect if there is a negative weight cycle in a directed and connected graph using the Bellman-Ford Algorithm.

### Overview

To solve this problem, we'll:
1. Use the Bellman-Ford Algorithm to find the shortest paths from a start vertex.
2. Check for negative weight cycles during the process.

### Steps

1. **Bellman-Ford Algorithm**:
   - **Initialize**:
     - `distance`: Dictionary to store the shortest distance from the start vertex. Initialize all distances to infinity, except the start vertex, which is set to 0.
   - **Relaxation**:
     - Relax all edges |V|-1 times (where V is the number of vertices).
     - For each vertex and its neighbors, update the distance if a shorter path is found.
   - **Cycle Detection**:
     - After |V|-1 iterations, check all edges one more time.
     - If any edge can still be relaxed (i.e., offers a shorter path), a negative weight cycle is present.
     - Return `None` if a negative weight cycle is detected, otherwise return the `distance` dictionary.

2. **IsNegativeWeightCyclePresent Function**:
   - **Run Bellman-Ford**:
     - Call `bellman_ford(adjacency_list, 0)` to get the distance dictionary or detect a cycle.
   - **Check for Cycle**:
     - If `bellman_ford` returns `None`, a negative weight cycle is present.
     - Return `True` if a cycle is detected, otherwise return `False`.

In [75]:
# Bellman-Ford Algorithm (standard implementation with cycle detection)
def bellman_ford(adjacency_list, start_vertex):
  distance = {vertex: float('inf') for vertex in adjacency_list}
  distance[start_vertex] = 0

  # Relax all edges |V|-1 times
  for _ in range(len(adjacency_list) - 1):
    for vertex in adjacency_list:
      for neighbor, weight in adjacency_list[vertex]:
        # same as dijkstra: you offer better distances to neighbors
        new_distance = distance[vertex] + weight
        if new_distance < distance[neighbor]:
          distance[neighbor] = new_distance

  # Check all edges one more time for cycle detection
  for vertex in adjacency_list:
    for neighbor, weight in adjacency_list[vertex]:
      # same as dijkstra: you offer better distances to neighbors
      new_distance = distance[vertex] + weight
      if new_distance < distance[neighbor]:
        return None  # NEW 😱

  return distance

In [76]:
def is_negative_weight_cycle_present(adjacency_list):
  distance = bellman_ford(adjacency_list, 0)
  print(f'{distance=}')
  return distance is None


edges = [(0, 1, 10), (0, 2, -5), (0, 3, 2), (3, 2, -5), (2, 1, -20), (1, 3, 10)]
is_negative_weight_cycle_present(create_directed_adjacency_list(edges)) == True

distance=None


True

In [77]:
# Test cases

edges = [(0, 1, 20), (1, 4, -10), (4, 0, 5), (0, 2, 10), (0, 3, 10), (2, 3, 20), (3, 4, 30), (3, 1, -20)]
is_negative_weight_cycle_present(create_directed_adjacency_list(edges)) == True

edges = [(0, 1, 20), (1, 2, 10), (2, 3, 20), (3, 1, -40)]
is_negative_weight_cycle_present(create_directed_adjacency_list(edges)) == True

edges = [(0, 1, 10), (1, 2, 20), (2, 3, -10), (3, 4, -5), (4, 5, -15), (5, 0, 5)]
is_negative_weight_cycle_present(create_directed_adjacency_list(edges)) == False

edges = [(0, 1, -10), (1, 2, -20), (2, 3, -20)]
is_negative_weight_cycle_present(create_directed_adjacency_list(edges)) == False

distance=None


True

distance=None


True

distance={0: 0, 1: 10, 2: 30, 3: 20, 4: 15, 5: 0}


True

distance={0: 0, 1: -10, 2: -30, 3: -50}


True

# PPA 1

### Problem (XYZ Courier)
Implement a class `XYZ_Courier` that calculates the minimum cost and shortest route for delivering items between cities using the Floyd-Warshall algorithm.

### Steps

1. **Floyd-Warshall Algorithm**:
   - **Initialize**:
     - `dist`: 2D dictionary to store shortest distances between every pair of vertices, initialized to infinity.
     - `predecessor`: 2D dictionary to store the predecessor of each vertex in the shortest path.
   - **Populate Initial Distances**:
     - Set `dist[vertex][vertex]` to 0.
     - Set `dist[vertex][neighbor]` to the weight of the edge and `predecessor[vertex][neighbor]` to the vertex itself.
   - **Update Distances**:
     - Iterate through all pairs `(i, j)` and update `dist[i][j]` if a shorter path is found via an intermediate vertex `k`.
     - Update `predecessor[i][j]` to `k`.

2. **XYZ_Courier Class**:
   - **Initialization**:
     - Compute `dist` and `predecessor` using `floyd_warshall`.
   - **Cost Method**:
     - Calculate and return the cost from `source` to `destination` using `dist[source][destination] * 5`.
   - **Route Method**:
     - Construct the path from `source` to `destination` using the `predecessor` dictionary.
     - Return the path in the correct order.

In [78]:
# Floyd-Warshall Algorithm (slightly modified for `predecessor`)
def floyd_warshall(adjacency_list):
  dist = {u: {v: float('inf') for v in adjacency_list} for u in adjacency_list}
  predecessor = {u: {v: None for v in adjacency_list} for u in adjacency_list}  # NEW 😱

  for vertex in adjacency_list:
    dist[vertex][vertex] = 0

  for vertex in adjacency_list:
    for neighbor, weight in adjacency_list[vertex]:
      dist[vertex][neighbor] = weight
      predecessor[vertex][neighbor] = vertex  # NEW 😱

  for k in adjacency_list:
    for i in adjacency_list:
      for j in adjacency_list:
        if dist[i][j] > dist[i][k] + dist[k][j]:
          dist[i][j] = dist[i][k] + dist[k][j]
          predecessor[i][j] = k  # NEW 😱

  return dist, predecessor

In [79]:
class XYZ_Courier:
  def __init__(self, adjacency_list):
    self.dist, self.predecessor = floyd_warshall(adjacency_list)

  def cost(self, source, destination):
    return self.dist[source][destination] * 5

  def route(self, source, destination):
    path = []
    while destination is not None:
      path.append(destination)
      destination = self.predecessor[source][destination]
    path.reverse()
    return path


edges = [(0, 1, 10), (0, 2, 50), (0, 3, 300), (5, 6, 45), (2, 1, 30),
         (6, 4, 37), (1, 6, 65), (2, 5, 76), (1, 3, 40), (3, 4, 60), (2, 4, 20)]
courier = XYZ_Courier(create_undirected_adjacency_list(edges))
print(courier.cost(0, 4))  # 300
print(courier.route(0, 4))  # [0, 1, 2, 4]

300
[0, 1, 2, 4]


# PPA 2

### Problem (Taxi Driver's Minimum Cost Route)

Find the minimum cost route for a taxi driver to get home by possibly picking up customers along the way. Costs may be _negative_ due to earnings from customers.

### Steps

  - **Note:** Dijkstra's algorithm is not used because it cannot handle _negative edge weights_.

1. **Bellman-Ford Algorithm**:
   - **Initialization**:
     - `distance`: Dictionary to store shortest distances from `start_vertex`, initialized to infinity.
     - `predecessor`: Dictionary to store the predecessor of each vertex in the shortest path.
   - **Relax Edges**:
     - Iterate through all edges |V|-1 times.
       - Update `distance` and `predecessor` if a shorter path is found.
   - **Result**:
     - Return `distance` and `predecessor` dictionaries.

2. **Construct Path**:
   - **Backtrack**:
     - Using `predecessor`, backtrack from `destination` to `source` to construct the path.
     - Reverse the path to get it in the correct order.

3. **min_cost Function**:
   - **Calculate Distance and Path**:
     - Use `bellman_ford` to get `distance` and `predecessor` from `source`.
     - Construct the path using `construct_path`.
   - **Return**:
     - Minimum cost and path from `source` to `destination`.

In [80]:
# Bellman-Ford Algorithm (slightly modified for `predecessor`)
def bellman_ford(adjacency_list, start_vertex):
  distance = {vertex: float('inf') for vertex in adjacency_list}
  distance[start_vertex] = 0
  predecessor = {vertex: None for vertex in adjacency_list}  # NEW 😱

  for _ in range(len(adjacency_list) - 1):
    for vertex in adjacency_list:  # S A B C D E F
      for neighbor, weight in adjacency_list[vertex]:
        # same as dijkstra: you offer better distances to neighbors
        new_distance = distance[vertex] + weight
        if new_distance < distance[neighbor]:
          distance[neighbor] = new_distance
          predecessor[neighbor] = vertex  # NEW 😱

  return distance, predecessor

In [81]:
def construct_path(predecessor, destination):
  path = []
  while destination is not None:
    path.append(destination)
    destination = predecessor[destination]
  path.reverse()
  return path

In [82]:
def min_cost(adjacency_list, source, destination):
  distance, predecessor = bellman_ford(adjacency_list, source)
  path = construct_path(predecessor, destination)
  return distance[destination], path


edges = [(0, 1, 1000), (0, 7, 800), (1, 5, 200), (2, 1, 100), (2, 3, 100), (3, 4, 300),
         (4, 5, 500), (5, 2, -200), (2, 4, -200), (6, 1, 400), (6, 5, 100), (7, 6, 100)]
min_cost(create_directed_adjacency_list(edges), 0, 1)

(900, [0, 7, 6, 5, 2, 1])

# Summary of Problems

### GrPA 1: FiberLink

**Problem**: Connect `n` cities with minimum fiber length.

**Solution Overview**:
- **Algorithm**: _**Kruskal's Algorithm**_
- **Steps**:
  1. Initialize MST and edge list.
  2. Sort edges by weight.
  3. Use Union-Find to avoid cycles and add edges to MST.
  4. Sum the weights of the MST edges to get the total fiber length required.

### GrPA 2: MinCostWalk

**Problem**: Find the minimum cost walk from node `S` to node `D` via node `V`.

**Solution Overview**:
- **Algorithm**: _**Dijkstra's Algorithm**_
- **Steps**:
  1. Implement Dijkstra's using a priority queue.
  2. Compute shortest paths from `S` to `V` and from `V` to `D`.
  3. Combine these paths to get the total minimum cost and the walk route.

### GrPA 3: IsNegativeWeightCyclePresent

**Problem**: Detect if there is a negative weight cycle in a directed and connected graph.

**Solution Overview**:
- **Algorithm**: _**Bellman-Ford Algorithm**_
- **Steps**:
  1. Initialize distances, setting the start vertex to 0.
  2. Relax all edges |V|-1 times.
  3. Check for negative weight cycles by relaxing edges one more time.

### PPA 1: XYZ Courier

**Problem**: Calculate the minimum cost and shortest route for delivering items between cities.

**Solution Overview**:
- **Algorithm**: _**Floyd-Warshall Algorithm**_
- **Steps**:
    1. Use three nested loops over all pairs of cities (`k`, `i`, `j`) to update distances.
    1. If a shorter path is found through an intermediate city `k`, update the distance and predecessor matrices.
    2. Define a class to return the minimum cost and the corresponding route between the specified cities.

### PPA 2: Taxi Driver's Minimum Cost Route

**Problem**: Find the minimum cost route for a taxi driver to get home, accounting for possible earnings from picking up customers along the way.

**Solution Overview**:
- **Algorithm**: _**Bellman-Ford Algorithm**_
- **Steps**:
  1. Use Bellman-Ford to compute shortest paths with possible negative weights.
  2. Construct the route using the predecessors from the Bellman-Ford algorithm.
  3. Return the minimum cost and the corresponding path.